Ensure that genai is installed

In [ ]:
!pip install google.generativeai

Load your API key and all necessary packages

In [2]:

###
# Load API Key File
###
import secret_api_key_file

###
# Load Google's LLM package
###
import google.generativeai as genai

###
# Import other stuff we'll need
###
import os
import pandas as pd
import json
import requests
import shutil
import time

###
# Store your API key
###

genai.configure(api_key=secret_api_key_file.gemini_api_key)

###
# Store the model we'll use for this: 1.5 Flash
# Full model list: https://ai.google.dev/gemini-api/docs/models/gemini
###

#Create a model
model = genai.GenerativeModel('gemini-1.5-flash')


Load the CSV file that contains the urls for all short form disclosures between 2014 and present day in 2024

In [3]:
# Load the CSV file
csv_file_path = r"C:\Users\Aidan\Documents\GitHub\fara_project_group_repo\short_form_14_to_24.csv"
short_form_df = pd.read_csv(csv_file_path)

Iterate through the first 100 urls and extract the filer's responses to sections 10-12.

Ensure the output doesn't contain the questions for each section -- we only want to preserve the unique text / the responses from each form.

Also ensure that the model isn't thrown off by the two-part questions of section 12 (the yes/no checkbox followed by the "if yes..." part).

In [ ]:
import os
import shutil
import time
import json
import pandas as pd
import requests

# Create a temporary folder if you didn't already run the prior step
temp_folder = os.path.join("./", "temp")
os.makedirs(temp_folder, exist_ok=True)

# Function to download a file from a URL
def download_file(url, folder):
    local_filename = os.path.join(folder, "temp_file.pdf")
    with requests.get(url, stream=True, verify=False) as r:
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)
    return local_filename

# List to store the results
results = []

# Iterate through the first 100 URLs
for url in short_form_df["URL"][:100]:
    # Delete any existing file in the temporary folder
    for filename in os.listdir(temp_folder):
        file_path = os.path.join(temp_folder, filename)
        if os.path.isfile(file_path):
            os.unlink(file_path)
    
    # Download the new file
    try:
        file_path = download_file(url, temp_folder)
        print(f"Downloaded and saved file from {url}")
        
        # Upload the file to Google's servers
        sample_short_form = genai.upload_file(file_path)
        time.sleep(1)  # Ensure at least 1-second delay between uploads
        
        # Define the prompt
        prompt = """
        Extract the text from the responses in sections 10-12 of the document. Don't add any additional commentary.

        Ignore the yes or no box in section 12. Only provide the text response to the question if there is one.

        Remove the following to preserve only the responses to the form's prompts and the numbers of the sections:

        - "List the foreign principal to whom you will render services in support of the primary registrant."
        - "Describe in detail all services which you will render to the foreign principal listed in Item 10 either directly, or through the primary registrant listed in Item 8."
        - "Do any of the above-described services include political activity as defined in Section l(o) of the Act* 1? If yes, describe separately and in detail such political activity. The response must include, but not be limited to, activities involving lobbying, promotion, perception management, public relations, economic development, and preparation or dissemination of informational materials."

        Format this all as structured json.

        Any empty sections should be left as "None"

        This is the structure of the json you should return:

        ```json
        [
        {
            "10.": "example text",
            "11.": "example text",
            "12.": "example text",
        },

        here's another example:

        ```json
        [
        {
            "10.": "example text",
            "11.": "example text",
            "12.": "None",
        },

        """
        
        # Run the file through Gemini
        response = model.generate_content([prompt, sample_short_form])
        time.sleep(1)  # Ensure at least 1-second delay between requests
        
        # Ensure response.text is as expected and strip code block markers
        response_text = response.text.strip()
        if response_text.startswith("```json"):
            response_text = response_text[7:-3].strip()  # Remove ```json and ```
        
        # Now attempt to parse the cleaned JSON
        try:
            # Parse JSON
            json_data = json.loads(response_text)
            
            # Convert JSON to DataFrame
            df = pd.DataFrame(json_data)

            # Add the URL column
            df.insert(0, "URL", url)
            
            # Display the DataFrame
            print("DataFrame created successfully.")
            display(df)  # Use display(df) in Jupyter notebooks to render nicely
            
            # Store the result
            results.append({
                "URL": url,
                "Output": df
            })
        except json.JSONDecodeError as e:
            print(f"JSON parsing error: {e}")
    except Exception as e:
        print(f"Failed to download or process file from {url}: {e}")

# Save the results to a CSV file
section_10_to_12_results_df = pd.concat([result["Output"] for result in results], ignore_index=True)
section_10_to_12_results_df_path = os.path.join("./", "section_10_to_12_results.csv")
section_10_to_12_results_df.to_csv(section_10_to_12_results_df_path, index=False)

print(f"Results saved to {section_10_to_12_results_df_path}")


Now, do a second pass focusing on extracting the text from any appendices in the file. We're doing this as a separate processing step from our work above to limit the number of tasks we're asking Gemini to do in any given prompt. This has significantly improved accuracy versus a single all-inclusive prompt.

In post-processing outside Gemini further down, we'll whittle this down to only sections 10-12. But The output is more accurate with the prompt below, which captures all appendices, than when we try to get it to search only for appendices pertaining to sections 10-12. Better to capture too much info and whittle down later, than not enough or inaccurate info.

In [ ]:
import os
import shutil
import time
import json
import pandas as pd
import requests

# Create a temporary folder if you didn't already run the prior step
temp_folder = os.path.join("./", "temp")
os.makedirs(temp_folder, exist_ok=True)

# Function to download a file from a URL
def download_file(url, folder):
    local_filename = os.path.join(folder, "temp_file.pdf")
    with requests.get(url, stream=True, verify=False) as r:
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)
    return local_filename

# List to store the results
results = []

# DataFrame to store extracted text and corresponding URLs
extracted_data = pd.DataFrame(columns=["URL", "Extracted_Text"])

# Iterate through the first 100 URLs
for url in short_form_df["URL"][:100]:
    # Delete any existing file in the temporary folder
    for filename in os.listdir(temp_folder):
        file_path = os.path.join(temp_folder, filename)
        if os.path.isfile(file_path):
            os.unlink(file_path)

    # Download the new file
    try:
        file_path = download_file(url, temp_folder)
        print(f"Downloaded and saved file from {url}")

        # Upload the file to Google's servers
        sample_short_form = genai.upload_file(file_path)

        # Define the prompt
        prompt = """
        Please extract all the text from the provided PDF file for the appendix page, which has the word "Appendix" in the page's header. Ensure that any handwritten content is also included to the best of your ability.

        Don't extract information from any page that doesn't have the word "Appendix" in the header.
        
        Structure it as JSON.

        If there is no appendix page, all columns should be blank but the column headers shown in the example output below MUST still be there.

        Here is an example output format:

        [
            {
                "Appendix response to Item 1": "None",
                "Appendix response to Item 2": "None",
                "Appendix response to Item 3": "Item 3: Residence Address. Broumana Main Road, Vermont Building, Ground Floor, 2753 Ain Saade, Metn LEBANON",
                "Appendix response to Item 4": "None",
                "Appendix response to Item 5": "None",
                "Appendix response to Item 6": "None",
                "Appendix response to Item 7": "None",
                "Appendix response to Item 8": "None",
                "Appendix response to Item 9": "None",
                "Appendix response to Item 10": "None",
                "Appendix response to Item 11": "None",
                "Appendix response to Item 12": "None",
                "Appendix response to Item 13": "None",
                "Appendix response to Item 14": "None",
                "Appendix response to Item 15": "None"
            }
        ]
        """

        # Run the file through Gemini
        response = model.generate_content([prompt, sample_short_form])

        # Ensure response.text is as expected and strip code block markers
        response_text = response.text.strip()
        if response_text.startswith("```json"):
            response_text = response_text[7:-3].strip()  # Remove ```json and ```

        # Now attempt to parse the cleaned JSON
        try:
            # Parse JSON
            json_data = json.loads(response_text)

            # Convert JSON to DataFrame
            df = pd.DataFrame(json_data)

            # Add the URL to the DataFrame and move it to the first column
            df.insert(0, "URL", url)

            # Display the DataFrame
            print("DataFrame created successfully.")
            display(df)  # Use display(df) in Jupyter notebooks to render nicely

            # Store the result
            results.append(df)
        except json.JSONDecodeError as e:
            print(f"JSON parsing error: {e}")
    except Exception as e:
        print(f"Failed to download or process file from {url}: {e}")

# Concatenate all results into a single DataFrame
results_appendices_df = pd.concat(results, ignore_index=True)

# Define the desired save path
save_folder = "./"
results_appendices_path = os.path.join(save_folder, "results_appendices.csv")

# Save the results to the specified folder
results_appendices_df.to_csv(results_appendices_path, index=False)

print(f"Results saved to {results_appendices_path}")


Join the two dataframes and only keep the relevant columns

In [ ]:
# Join results_appenices_df with results_10_to_12_df on URL
merged_df = pd.merge(results_appendices_df, section_10_to_12_results_df, on="URL", how="inner")

# Keep only the specified columns in the final DataFrame
final_columns = ["URL", "10.", "11.", "12.", "Appendix response to Item 10", "Appendix response to Item 11", "Appendix response to Item 12"]
final_df = merged_df[final_columns]

# Display the final DataFrame
print("Final DataFrame:")
display(final_df)

# Save the final DataFrame to a CSV file in the specified directory
final_results_path = r"C:\Users\Aidan\Documents\GitHub\fara_project_group_repo\final_results.csv"
final_df.to_csv(final_results_path, index=False)

print(f"Final results saved to {final_results_path}")
